This notebook illustrates the code used in the paper ```U. Dobramysl, D. Holcman, Mixed analytical-stochastic simulation method for the recovery of a Brownian gradient source from probability fluxes to small windows, Journal of Computational Physics 355 (2018)```.

To run this notebook, you need a Python 3 installation (e.g. Anaconda) and the python packages `cython`, `numpy`, `scipy`, `matplotlib`, `seaborn`, `multiprocess` and `tqdm`. To install them, simply run:

```pip install cython numpy scipy matplotlib seaborn multiprocess tqdm```

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from utils import json_cache_write, json_cache_read, send_parallel_jobs
import pyximport; pyximport.install()
from receptor_fluxes_halfspace import SimulationDomain
rc("figure", figsize=(8,6))
sns.set_context("poster")
sns.set_style("ticks"); None

# Simulation of particle fluxes to receptors at steady state - Half-space

In [ ]:
def run_simulation(parameters):
    """Perform a single realisation of the simulation.
    
    Args:
        source_distance (float): Distance to the source x.
        thetas (list|floats): Receptor angles.
        nr_particles (int): Number of particles to release.
    Returns:
        Receptor fluxes.
    """
    from receptor_fluxes_halfspace import SimulationDomain
    
    params = {
        "receptor_distance": 10.,
        "nr_particles": 1e4,
        "receptor_epsilon": 1.,
    }
    params.update(parameters)
    
    receptor_positions = [-params["receptor_distance"]/2., params["receptor_distance"]/2.]
    
    source_position = (
        params["source_distance"]*np.cos(params["source_angle"]),
        params["source_distance"]*np.sin(params["source_angle"]))
    
    sd = SimulationDomain(receptor_positions, receptor_epsilon=params['receptor_epsilon'],
                          source_position=source_position)
    for _ in range(int(params["nr_particles"])):
        sd.release_particle()
    return {"parameters": params, "results": {"normalized_receptor_fluxes": sd.receptor_fluxes()[1].tolist()}}

## Simulation Results

In [ ]:
import json
import gzip
def json_cache_write(name, data):
    with gzip.open(name + '.json.gz', 'wt', encoding='utf8') as f:
        json.dump(data, f)
        
def json_cache_read(name):
    with gzip.open(name + '.json.gz', 'rt', encoding='utf8') as f:
        return json.load(f)

In [ ]:
from multiprocess import Pool
from tqdm.auto import tqdm

def send_parallel_jobs(name, todo):
    try:
        return json_cache_read(name)
    except (IOError, ValueError):
        pass

    with Pool() as p:
        results = [r for r in tqdm(p.imap_unordered(run_simulation, todo), total=len(todo), smoothing=0)]
    
    json_cache_write(name, results)
    return results

In [ ]:
def HalfspaceFlux(L, theta, x1, x2, eps):
    y = np.vstack([L*np.cos(theta), L*np.sin(theta)]).swapaxes(0,1)
    def norm(v):
        v = np.atleast_2d(v)
        return (v**2).sum(axis=1)**0.5
    return (np.log(norm(x2[np.newaxis]-y)/eps)/
            (np.log(norm(x1[np.newaxis]-y)/eps)+
             np.log(norm(x2[np.newaxis]-y)/eps)))

In [ ]:
def HalfspaceFlux2(L, theta, x1, x2, eps, alpha):
    y = np.vstack([L*np.cos(theta), L*np.sin(theta)]).swapaxes(0,1)
    def norm(v):
        if len(v.shape)==1:
            return (v**2).sum()**0.5
        return (v**2).sum(axis=1)**0.5
    return (0.5+alpha*np.log(norm(y-x1)/norm(y-x2))/np.log(eps/norm(x1-x2)))

In [ ]:
from matplotlib.lines import Line2D
from scipy.optimize import leastsq
def plot_fluxes(name, fix=[], legend=None, axlabel="", labelpos=0.3):
    from collections import defaultdict
    from scipy.interpolate import interp1d
    data = defaultdict(lambda : defaultdict(list))
    for res in json_cache_read(name):
        params = res["parameters"]
        fluxes = res["results"]["normalized_receptor_fluxes"]
        data[(params['source_distance'], params['receptor_distance'])][params['source_angle']].append(fluxes[1])
    maxf = 0.0
    minf = 1.0
    mstyles = iter(['o', 'v', '<', '>'])
    styles = iter(['-', '--', '-.', ':'])
    if not isinstance(labelpos, list):
        labelpos = len(data)*[labelpos]
    handles = []
    labels = []
    for j, x in enumerate(sorted(data)):
        d = data[x]
        theta = np.array(sorted(d))
        fluxes = np.array([np.mean(d[t]) for t in theta])
        maxf = max(max(fluxes), maxf)
        minf = min(min(fluxes), minf)
        for f in fix:
            fluxes[f] = float("NaN")
        mark = next(mstyles)
        lstyle = next(styles)
        l = plt.plot(theta, fluxes, mark, color='black', label="$L=%g$" % x[0])[0]
        recpos = np.array([(0, -x[0]/2.), (0, x[0]/2.)])
        ttheta = np.linspace(theta[0], theta[-1], 100)
        eps = params['receptor_epsilon']
        
        fitfunc = lambda alpha: fluxes[1:-1]-HalfspaceFlux2(x[0], theta[1:-1]-np.pi/2.,  recpos[0], recpos[1], eps/2., alpha)
        p = leastsq(fitfunc, (0.5,))
        print(x[0], p)
        
        plt.plot(ttheta, HalfspaceFlux2(x[0], ttheta-np.pi/2.,  recpos[0], recpos[1], eps/2., p[0][0]), color='black', ls=lstyle)
        
        handles.append(Line2D([0, 1], [0,0], color='black', ls=lstyle, marker=mark))
        labels.append("$L=%g$" % x[0])
    maxt = max(theta)
    plt.xlabel(r"$\theta$")
    plt.ylabel(r"$J_2/(J_1+J_2)$")
    plt.xticks([0, np.pi/4, np.pi/2, np.pi*3/4., np.pi, np.pi*5/4., np.pi*3/2., np.pi*7/4., 2*np.pi],
               ["$0$", r"$\pi/4$", r"$\pi/2$", r"$3\pi/4$", r"$\pi$", r"$5\pi/4$", r"$3\pi/2$", r"$7\pi/8$", r"$2\pi$"])
    plt.axis(xmax=1*maxt, ymin=min(0.45, 0.95*minf), ymax=max(0.51, maxf))
    if legend is not None:
        plt.legend(handles, labels, loc=legend)
    plt.text(0.05, 0.05, axlabel, transform=plt.gca().transAxes)
    plt.axis('tight')

In [ ]:
todo = sum([[{"source_distance": x, "receptor_distance": 1., "receptor_epsilon": 0.1,
              "source_angle":angle, 'nr_particles': 1e3} for angle in np.linspace(0, np.pi, 10)]
            for x in [1.0, 2.0, 5.0, 10.0]], [])
results = send_parallel_jobs("halfspace", run_simulation, todo)
plot_fluxes("halfspace", legend="upper center")

In [ ]:
todo = sum([[{"source_distance": x, "receptor_distance": 1., "receptor_epsilon": 0.05,
              "source_angle":angle, 'nr_particles': 1e4} for angle in np.linspace(0, np.pi, 10)]
            for x in [1.0, 2.0, 5.0, 10.0]], [])
results = send_parallel_jobs("halfspace_smalleps", run_simulation, todo)
plot_fluxes("halfspace_smalleps", legend="upper center")

In [ ]:
todo = sum([[{"source_distance": x, "receptor_distance": 1., "receptor_epsilon": 0.01,
              "source_angle":angle, 'nr_particles': 1e4} for angle in np.linspace(0, np.pi, 10)]
            for x in [1.0, 2.0, 5.0, 10.0]], [])
results = send_parallel_jobs("halfspace_smallsmalleps", run_simulation, todo)
plot_fluxes("halfspace_smallsmalleps", legend="upper center")

In [ ]:
todo = sum([[{"source_distance": x, "receptor_distance": 1., "receptor_epsilon": 0.25,
              "source_angle":angle, 'nr_particles': 1e4} for angle in np.linspace(0, np.pi, 10)]
            for x in [1.0, 2.0, 5.0, 10.0]], [])
results = send_parallel_jobs("halfspace_largeeps", run_simulation, todo)
plot_fluxes("halfspace_largeeps", legend="upper center")

In [ ]:
todo = sum([[{"source_distance": x, "receptor_distance": 2., "receptor_epsilon": 0.1,
              "source_angle":angle, 'nr_particles': 1e4} for angle in np.linspace(0, np.pi, 10)]
            for x in [1.0, 2.0, 5.0, 10.0]], [])
results = send_parallel_jobs("halfspace_largedx", run_simulation, todo)
plot_fluxes("halfspace_largedx", legend="upper center")